# Import libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv

Matplotlib is building the font cache; this may take a moment.


# Import air quality data

In [ ]:
pp_airquality = np.load('pp_airquality.npz', allow_pickle=True)

pp_airquality = pp_airquality['X']

NpzFile 'pp_airquality.npz' with keys: X


In [5]:
print(pp_airquality)

[['AL' 'Baldwin' 274.5 ... 181.5 93.0 0.0]
 ['AL' 'Barbour' 32.0 ... 0.0 32.0 0.0]
 ['AL' 'Clay' 278.5 ... 183.0 95.5 0.0]
 ...
 ['WY' 'Sublette' 335.5 ... 316.0 15.5 0.0]
 ['WY' 'Sweetwater' 365.5 ... 0.0 0.0 365.5]
 ['WY' 'Teton' 342.5 ... 311.0 26.0 1.5]]


# Import natality data

In [76]:
pp_nat = np.load('/Users/jadeg/Documents/UW/Spring/Erdos/X_final_no_AQI_lbw.npz', allow_pickle=True)
pp_nat = pp_nat['X']

In [77]:
print(pp_nat)

[[2 2 2 ... 3 'Anchorage Munic' 'AK']
 [2 1 1 ... 3 'Anchorage Munic' 'AK']
 [2 1 1 ... 3 'Anchorage Munic' 'AK']
 ...
 [0 4 2 ... 3 'Kings' 'NY']
 [0 1 1 ... 3 'Queens' 'NY']
 [0 4 2 ... 3 'Queens' 'NY']]


# add air quality as a column to every county name and state that matches it, count and drop all rows that do not have a county in the air quality file

## convert to dataframes

In [78]:
aq_df = pd.DataFrame(pp_airquality)
mat_df = pd.DataFrame(pp_nat)

## pull out state and county names from air quality

In [79]:
aq_df[0] = aq_df[0].str.strip().str.upper()   # state abbrev
aq_df[1] = aq_df[1].str.strip().str.lower()   # county name

# find in maternal df
mat_df.iloc[:, -1] = mat_df.iloc[:, -1].str.strip().str.upper()  # state abbrev (last col)
mat_df.iloc[:, -2] = mat_df.iloc[:, -2].str.strip().str.lower()  # county name (second to last)


In [80]:
aq_df.columns = ['state', 'county'] + [f'aq{i}' for i in range(1, aq_df.shape[1] - 1)]
mat_df.columns = [f'mat{i}' for i in range(mat_df.shape[1] - 2)] + ['county', 'state']

In [81]:
print(aq_df['county'])

0          baldwin
1          barbour
2             clay
3          colbert
4           dekalb
           ...    
1097        platte
1098      sheridan
1099      sublette
1100    sweetwater
1101         teton
Name: county, Length: 1102, dtype: object


In [82]:
print(mat_df)

        mat0 mat1 mat2 mat3 mat4 mat5 mat6 mat7 mat8 mat9  ... mat69 mat70  \
0          2    2    2   22    1    2    2    1    1    1  ...     1     2   
1          2    1    1   32    9    9    9    2    2    3  ...     2     2   
2          2    1    1   20    9    9    9    2    2    3  ...     2     1   
3          2    1    1   34    9    9    9    2    2    3  ...     2     1   
4          2    1    1   27    9    9    9    2    2    3  ...     2     1   
...      ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   ...   ...   
3450061    0    4    2   21    1    0    0    1    1    2  ...     2     2   
3450062    0    4    2   37    1    0    0    1    1    2  ...     9     9   
3450063    0    4    2   32    1    0    0    1    1    1  ...     2     2   
3450064    0    1    1   16    1    0    0    1    2    1  ...     2     2   
3450065    0    4    2   36    1    0    0    1    1    1  ...     9     9   

        mat71 mat72 mat73 mat74 mat75 mat76           county st

# drop other maternal outcome data

In [83]:
labels = ['ocntypop', 'ubfacil', 'bfacil3', 'mager', 'msa_pop', 'rcnty_pop', 'rcity_pop', 'metrores', 'rectype', 'mrace', 'mracehisp', 'mar', 'dmeduc', 'meduc_rec', 'fagerpt', 'ufagecomb', 'fracerec', 'fracehisp', 'frace', 'priorlive', 'priordead', 'priorterm', 'lbo', 'tbo', 'mpcb', 'uprevis', 'wtgain', 'u_apncu', 'tobuse', 'cigs', 'drinks', 'urf_anemia', 'urf_card', 'urf_lung', 'urf_diab', 'urf_gen', 'urf_hydra', 'urf_hemo', 'urf_chyper', 'urf_phyper', 'urf_eclam', 'urf_incerv', 'urf_pre4000', 'urf_preterm', 'urf_renal', 'urf_rh', 'urf_uterine', 'urf_other', 'uop_amnio', 'uop_monit', 'uop_induc', 'uop_stiml', 'uop_tocol', 'uop_ultra', 'uop_other', 'uld_febr', 'uld_meco', 'uld_ruptr', 'uld_abrup', 'uld_prepla', 'uld_excbl', 'uld_seiz', 'uld_precip', 'uld_prolg', 'uld_dysfn', 'uld_breech', 'uld_cephal', 'uld_cord', 'uld_anest', 'uld_distr', 'uld_other', 'dmeth_rec', 'attend', 'dplural', 'sex', 'gestrec3', 'bwtr4', 'cntname','state']
print(len(labels))
exc_labels =  ['urf_anemia', 'urf_eclam', 'gestrec3', 'bwtr4']
exc_indexes = [i for i, name in enumerate(labels) if name in exc_labels]
eclam_idx = [i for i, name in enumerate(labels) if name=='urf_eclam']
lw_idx = [i for i, name in enumerate(labels) if name=='bwtr4']
print(eclam_idx)
print(lw_idx)
print(exc_indexes)

79
[40]
[76]
[31, 40, 75, 76]


In [84]:
# cols_to_drop = ['mat31', 'mat75','mat76']
cols_to_drop = ['mat31', 'mat75','mat40']
matdf_dropped = mat_df.drop(columns=cols_to_drop)
print(matdf_dropped)

        mat0 mat1 mat2 mat3 mat4 mat5 mat6 mat7 mat8 mat9  ... mat68 mat69  \
0          2    2    2   22    1    2    2    1    1    1  ...     2     1   
1          2    1    1   32    9    9    9    2    2    3  ...     2     2   
2          2    1    1   20    9    9    9    2    2    3  ...     2     2   
3          2    1    1   34    9    9    9    2    2    3  ...     2     2   
4          2    1    1   27    9    9    9    2    2    3  ...     2     2   
...      ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   ...   ...   
3450061    0    4    2   21    1    0    0    1    1    2  ...     2     2   
3450062    0    4    2   37    1    0    0    1    1    2  ...     9     9   
3450063    0    4    2   32    1    0    0    1    1    1  ...     2     2   
3450064    0    1    1   16    1    0    0    1    2    1  ...     2     2   
3450065    0    4    2   36    1    0    0    1    1    1  ...     9     9   

        mat70 mat71 mat72 mat73 mat74 mat76           county st

In [85]:
merged = matdf_dropped.merge(aq_df, on=['state', 'county'], how='left')
merged = merged.drop(columns=['state', 'county'])

In [86]:
print(merged)

        mat0 mat1 mat2 mat3 mat4 mat5 mat6 mat7 mat8 mat9  ...  aq6  aq7  \
0          2    2    2   22    1    2    2    1    1    1  ...  NaN  NaN   
1          2    1    1   32    9    9    9    2    2    3  ...  NaN  NaN   
2          2    1    1   20    9    9    9    2    2    3  ...  NaN  NaN   
3          2    1    1   34    9    9    9    2    2    3  ...  NaN  NaN   
4          2    1    1   27    9    9    9    2    2    3  ...  NaN  NaN   
...      ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
3450061    0    4    2   21    1    0    0    1    1    2  ...  0.0  0.0   
3450062    0    4    2   37    1    0    0    1    1    2  ...  0.0  0.0   
3450063    0    4    2   32    1    0    0    1    1    1  ...  0.0  0.0   
3450064    0    1    1   16    1    0    0    1    2    1  ...  0.0  0.0   
3450065    0    4    2   36    1    0    0    1    1    1  ...  0.0  0.0   

           aq8   aq9  aq10   aq11  aq12  aq13   aq14 aq15  
0          NaN   NaN   NaN 

# count unmatched rows

In [87]:
missing_matches = merged[[col for col in merged.columns if col.startswith('aq')]].isnull().any(axis=1).sum()
print(f"Number of unmatched rows: {missing_matches}")

Number of unmatched rows: 203148


# Drop rows with missing air quality data

In [90]:
merged_clean = merged.dropna(subset=[col for col in merged.columns if col.startswith('aq')])

In [91]:
print(merged_clean)
print(np.shape(merged_clean))

        mat0 mat1 mat2 mat3 mat4 mat5 mat6 mat7 mat8 mat9  ...  aq6  aq7  \
5609       1    1    1   18    1    1    3    1    1    2  ...  0.0  0.0   
5610       3    1    1   18    9    9    9    2    2    2  ...  0.0  0.0   
5611       3    1    1   30    1    3    3    1    1    2  ...  0.0  0.0   
5612       3    1    1   20    1    3    3    1    1    2  ...  0.0  0.0   
5613       2    1    1   30    1    2    3    1    1    2  ...  0.5  0.0   
...      ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
3450061    0    4    2   21    1    0    0    1    1    2  ...  0.0  0.0   
3450062    0    4    2   37    1    0    0    1    1    2  ...  0.0  0.0   
3450063    0    4    2   32    1    0    0    1    1    1  ...  0.0  0.0   
3450064    0    1    1   16    1    0    0    1    2    1  ...  0.0  0.0   
3450065    0    4    2   36    1    0    0    1    1    1  ...  0.0  0.0   

           aq8    aq9  aq10   aq11  aq12   aq13   aq14  aq15  
5609     171.5  103.0  7

# Drop column with eclampsia, save as Y

In [92]:
#df_eclam = merged_clean['mat40']
#Y_eclam = df_eclam.to_numpy()
#np.savez_compressed('Yeclam.npz', X=Y_eclam)
df_lw = merged_clean['mat76']
Y_lw = df_lw.to_numpy()
np.savez_compressed('Y_lw.npz',X=Y_lw)

In [93]:
#merged_clean_noeclam = merged_clean.drop(columns=['mat40'])
merged_clean_noeclam = merged_clean.drop(columns=['mat76'])

In [94]:
print(np.shape(Y_lw))

(3246918,)


# convert to numpy

In [95]:
X_mat_aqi = merged_clean_noeclam.to_numpy()

In [96]:
print(X_mat_aqi)
print(np.shape(X_mat_aqi))

[[1 1 1 ... 33.5 288.0 39.0]
 [3 1 1 ... 167.5 115.5 0.0]
 [3 1 1 ... 167.5 115.5 0.0]
 ...
 [0 4 2 ... 0.0 112.5 4.5]
 [0 1 1 ... 32.0 248.0 0.0]
 [0 4 2 ... 32.0 248.0 0.0]]
(3246918, 88)


# Make X dataframe with just mat, no aqi

In [97]:
mat_only = merged_clean.drop(columns=[col for col in merged_clean.columns if col.startswith('aq')])

In [98]:
X_mat_only = mat_only.to_numpy()

In [99]:
print(X_mat_only)
print(np.shape(X_mat_only))

[[1 1 1 ... 1 0 3]
 [3 1 1 ... 1 1 3]
 [3 1 1 ... 1 1 3]
 ...
 [0 4 2 ... 1 0 3]
 [0 1 1 ... 1 1 3]
 [0 4 2 ... 1 1 3]]
(3246918, 74)


# Save as .npz

In [100]:
np.savez_compressed('X_mat_aqi_lw.npz', X=X_mat_aqi)

In [101]:
np.savez_compressed('X_mat_only_lw.npz', X=X_mat_aqi)